https://github.com/walsvid/CoordConv/blob/master/coordconv.py
https://medium.com/analytics-vidhya/encoding-time-series-as-images-b043becbdbf3
https://eng.uber.com/coordconv/
https://towardsdatascience.com/reading-charts-with-convolutional-neural-networks-cbaabdd5f478

# Intro

In this notebook we build a CNN classifier for the problem

In [1]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from coordconv import CoordConv2d

import copy
import argparse
import os
import logging
import sys
from tqdm import tqdm
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

from ipywidgets import IntProgress
from IPython.display import display


In [2]:
# Create a custom data loader for the train, test, & validation data
class NumericalAndImageDataset(Dataset):
    def __init__(self, overview_file: str, transform=None):
        """
        Initialize this dataloader
        :param overview_file: location of the overview file
        :param transform: transformer for the images
        """
        self.overview= pd.read_csv(overview_file)
        self.transform = transform


    def __len__(self):
        return len(self.overview.index)


    def __getitem__(self, idx):
        img_path_1m = self.overview["1_month_img"].iloc[idx]
        img_path_6m = self.overview["6_month_img"].iloc[idx]
        img_path_12m = self.overview["12_month_img"].iloc[idx]

        img_1m = Image.open(img_path_1m).convert('RGB')
        img_6m = Image.open(img_path_6m).convert('RGB')
        img_12m = Image.open(img_path_12m).convert('RGB')

        num_features = torch.tensor(self.overview[[
            "1_month_return", "6_month_return", "12_month_return",
            "1_month_volatility", "6_month_volatility", "12_month_volatility"
        ]].iloc[idx].values)

        label = self.overview.label.iloc[idx]

        if self.transform:
            img_1m = self.transform(img_1m)
            img_6m = self.transform(img_6m)
            img_12m = self.transform(img_12m)

        return img_1m, img_6m, img_12m, num_features, label

In [3]:
class OneImageNet(nn.Module):
    def __init__(self):
        super(OneImageNet, self).__init__()

        self.image_1_features = nn.Sequential(
            CoordConv2d(3, 8, kernel_size=4, stride=1, with_r=True, use_cuda=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # nn.Dropout(),
            nn.Conv2d(8, 16, kernel_size=8),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # nn.Dropout(),
            nn.Conv2d(16, 32, kernel_size=8),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # nn.Dropout(),
        )

        self.fc = nn.Sequential(
            nn.Linear(32 * 48 * 48, 128),
            nn.ReLU(inplace=True),
            # nn.Dropout(),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            # nn.Dropout(),
            nn.Linear(64, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 3)
        )

    def forward(self, x):
        step1 = self.image_1_features(x)
        # print(step1.shape)
        step1 = step1.view(step1.size(0), -1)
        res = self.fc(step1)
        return res

In [4]:
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0
    running_corrects = 0
    running_total = 0

    for _, img_6ms, _, _, labels in test_loader:
        outputs = model(img_6ms)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        running_loss += loss.item() * img_6ms.size(0)
        running_corrects += torch.sum(preds == labels.data)

        running_total += len(img_6ms)

    total_loss = running_loss / running_total
    total_acc = running_corrects.double() / running_total

    print("Test Loss: {}".format(total_loss))
    print("Test Accuracy: {}".format(total_acc))

    return total_loss, total_acc

In [5]:
def train(model, train_loader, validation_loader, criterion, optimizer, epochs):
    datasets = {'train':train_loader, 'valid':validation_loader}
    #log = Report(epochs)

    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []

    for epoch in range(epochs):
        print("Epoch: " + str(epoch))

        f = IntProgress(min=0, max=5000) # instantiate the bar
        display(f) # display the bar

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
                running_loss = 0.0
                running_corrects = 0
                running_total = 0

                for pos, (_, img_6ms, _, _, labels) in enumerate(datasets[phase]):
                    f.value += len(img_6ms)

                    outputs = model(img_6ms)
                    loss = criterion(outputs, labels)

                    if phase=='train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    _, preds = torch.max(outputs, 1)
                    running_loss += loss.item() * img_6ms.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    running_total += len(img_6ms)

                train_losses.append(running_loss / running_total)
                train_accs.append(running_corrects / running_total)

                print("Train Loss: {}".format(running_loss / running_total))
                print("Train Accuracy: {}".format(running_corrects / running_total))

            if phase == "valid":
                val_loss, val_acc = test(model, datasets[phase], criterion)
                val_losses.append(val_loss)
                val_accs.append(val_acc)

    return model, train_losses, val_losses, train_accs, val_accs

In [6]:
def create_data_loaders(batch_size):
    train_data_path = "ModelData/obs_train.csv"
    test_data_path = "ModelData/obs_test.csv"
    val_data_path = "ModelData/obs_val.csv"

    train_transform = transforms.Compose([
        # transforms.RandomResizedCrop((224, 224)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

    test_transform = transforms.Compose([
        # transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    train_data = NumericalAndImageDataset(
        overview_file=train_data_path,
        transform=train_transform
    )
    train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

    test_data = NumericalAndImageDataset(
        overview_file=test_data_path,
        transform=test_transform
    )
    test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

    val_data = NumericalAndImageDataset(
        overview_file=val_data_path,
        transform=test_transform
    )
    val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

    return train_data_loader, test_data_loader, val_data_loader

In [7]:
batch_size = 128
learning_rate = 0.00075
epochs = 20

train_loader, test_loader, val_loader = create_data_loaders(batch_size)
model = OneImageNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Use adaptive momentum optimizer

print("Starting Model Training")

model, train_losses, val_losses, train_accs, val_accs = train(model, train_loader, val_loader, criterion, optimizer, epochs)

print("Train Losses:")
print(train_losses)
print("Validation Losses:")
print(val_losses)
print("Train Accuracies:")
print(train_accs)
print("Validation Accuracies:")
print(val_accs)

print("Start Model Testing")

test_loss, test_acc = test(model, test_loader, criterion)

print('saved')

Starting Model Training
Epoch: 0


IntProgress(value=0, max=5000)

Train Loss: 1.0324744725570696
Train Accuracy: 0.4246082305908203
Test Loss: 1.0218131300353843
Test Accuracy: 0.4531977975434138
Epoch: 1


IntProgress(value=0, max=5000)

Train Loss: 1.01287811913767
Train Accuracy: 0.4563744068145752
Test Loss: 1.0107286660058272
Test Accuracy: 0.4531977975434138
Epoch: 2


...

Train Loss: 1.003949031543045
Train Accuracy: 0.4593392610549927
Test Loss: 1.0150222916918152
Test Accuracy: 0.4557390936044049
Epoch: 7


IntProgress(value=0, max=5000)

Train Loss: 1.00710390109927
Train Accuracy: 0.4551037549972534
Test Loss: 1.0040246208104882
Test Accuracy: 0.45235069885641677
Epoch: 8


IntProgress(value=0, max=5000)

KeyboardInterrupt: 